# Type Hints - how do they behave?

In [1]:
from abc import ABC, abstractmethod
from typing import Any, Generic, TypeVar, get_type_hints

In [2]:
__file__ = "type-hints.ipynb"

def check():
    !nbqa mypy {__file__}

In [3]:
check()

Success: no issues found in 1 source file


## Hints are not inherited by overridden methods

In [4]:
class Base1(ABC):
    @abstractmethod
    def method(self, arg: int) -> str:
        raise NotImplementedError

class Derived1(Base1):
    def method(self, arg):
        return str(arg)

assert Derived1().method(3) == "3"

In [5]:
check()

Success: no issues found in 1 source file


In [6]:
get_type_hints(Base1.method)

{'arg': int, 'return': str}

In [7]:
get_type_hints(Derived1.method)

{}

## Hints must be compatible between base & derived

In [8]:
class Base2(ABC):
    @abstractmethod
    def method1(self, arg: int) -> str:
        raise NotImplementedError
        
    @abstractmethod
    def method2(self, arg: Any) -> Any:
        raise NotImplementedError

class Derived2a(Base2):
    def method1(self, arg):
        return str(arg)

class Derived2b(Base2):
    def method1(self, arg: Any) -> str:
        return str(arg)

"""
# ERROR: This violates the Liskov substitution principle
class Derived2c(Base2):
    def method1(self, arg: str) -> Any:
        #                  ^^^
        return str(arg)
"""

class Derived2d(Base2):
    def method2(self, arg: Any) -> str:
        return str(arg)
    
class Derived2e(Base2):
    def method2(self, arg: int) -> Any:
        return str(arg)

In [9]:
check()

Success: no issues found in 1 source file


## Existential types

We can simulate existential types with generics and type variables.

In [10]:
T = TypeVar("T")

class BaseChecker(ABC, Generic[T]):
    @abstractmethod
    def check(self, x: T) -> bool:
        raise NotImplementedError

class ListChecker(BaseChecker[list]):
    pass

class IntChecker(BaseChecker[int]):
    def check(self, x: int) -> bool:
        return x > 0

class StrChecker(BaseChecker[str]):
    def check(self, x: str) -> bool:
        return " " not in x
    
assert IntChecker().check(1) is True
assert StrChecker().check("foo") is True

In [11]:
check()

Success: no issues found in 1 source file


In [12]:
get_type_hints(ListChecker.check)

{'x': ~T, 'return': bool}

In [13]:
get_type_hints(IntChecker.check)

{'x': int, 'return': bool}

In [14]:
get_type_hints(StrChecker.check)

{'x': str, 'return': bool}